In [ ]:
import pandas as pd
import numpy as np


In [ ]:
finale = ["data/combined_explicit.csv"]

In [ ]:
##create for loop to cast csv into the correct dtype

for i in finale:
    dtypes = {"Position":"int64","Track Name":"object","Artist" :"object","Streams": "int64","region":"object","spotify_id" :"object","temp":"int64","cloud":"int64","humidity": "int64","snow":"int64","rain":"int64","danceability":"float64","energy":"float64","key":"float64","loudness":"float64","mode":"float64","speechiness":"float64","acousticness":"float64","instrumentalness":"float64","liveness":"float64","valence":"float64","tempo":"float64","type":"object","id":"object","uri":"object","track_href":"object","analysis_url":"object","duration_ms":"float64","time_signature":"float64","audio_features":"object", "explicit": "float64"}
    date_cols = ['date']
    df = pd.read_csv(i, sep=',',header=0,  dtype=dtypes, parse_dates=date_cols)
    
    

In [ ]:
##Check dtypes

df.dtypes

In [ ]:
## delete Turkey

df = df[~df.region.str.contains("tr")]

## add month

df['month'] = df['date'].dt.month

## add constant for lin_reg

df = df.assign(const=1)

In [ ]:
## change regions into ISO-3 for mapping

df["region"] = df["region"].str.replace("at","AUT")
df["region"] = df["region"].str.replace("be","BEL")
df["region"] = df["region"].str.replace("ch","CHE")
df["region"] = df["region"].str.replace("de","DEU")
df["region"] = df["region"].str.replace("dk","DNK")
df["region"] = df["region"].str.replace("es","ESP")
df["region"] = df["region"].str.replace("fi","FIN")
df["region"] = df["region"].str.replace("fr","FRA")
df["region"] = df["region"].str.replace("gr","GRC")
df["region"] = df["region"].str.replace("hu","HUN")
df["region"] = df["region"].str.replace("ie","IRL")
df["region"] = df["region"].str.replace("is","ISL")
df["region"] = df["region"].str.replace("it","ITA")
df["region"] = df["region"].str.replace("nl","NLD")
df["region"] = df["region"].str.replace("no","NOR")
df["region"] = df["region"].str.replace("pl","POL")
df["region"] = df["region"].str.replace("pt","PRT")
df["region"] = df["region"].str.replace("ro","ROU")
df["region"] = df["region"].str.replace("se","SWE")
df["region"] = df["region"].str.replace("sk","SVK")


In [ ]:
## check for NaN's
isna = df.isna().sum()
df.loc[isna]

In [ ]:
## check if theres any duplicates

df.duplicated(subset=None, keep='first').sum()

In [ ]:
## drop all features that aren't valuable or more then 90% NaN

dropped=['audio_features','duration_ms', 'time_signature', 'track_href', 'uri', 'analysis_url', 'type', 'id', 'mode']

In [ ]:
df = df.drop(columns=dropped)

In [ ]:
## make columns lowercase for a clean look

df = df.rename(columns={"Artist": "artist", "Position": "position", "Track Name": "track_name", "Streams": "streams"})

In [ ]:
feature = ['acousticness','danceability', 'energy', 'instrumentalness', 'key','liveness', 'loudness', 'speechiness', 'temp', 'tempo', 'valence']
country = ['is','se','fi','se','at','sk','ro','no','ch','nl','de','be','fr','ie','pl','hu','dk','es','pt','gr','it']

In [ ]:
## used a weighted mean by streams so that each measurement is relative the country 


for i in feature:
    for j in country:
    # Define a lambda function to compute the weighted mean:
        
        wm = lambda x: np.average(x, weights=df.loc[x.index, "streams"])
    
    # Define a dictionary with the functions to apply for a given column:
        f = { i : {'weighted_mean' : wm} }
        
    
    # Groupby and aggregate with your dictionary:
        weight = df.groupby(["month","region"]).agg(f)
        
    # Export to csv
        weight.to_csv(f"{j}_{i}", index=True)
        
        

In [ ]:
## round all numbers to 2 decimals places

df = df.round(2)

In [ ]:
df.columns

In [ ]:
## rearrange columns for ease of use

cols = ['region', 'date', 'month','spotify_id', 'artist', 'track_name','position', 'streams','danceability', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'speechiness', 'acousticness', 'tempo', 'valence', 'explicit','temp', 'rain', 'snow',  'cloud', 'humidity', 'const']

df = df[cols] 

In [ ]:
## final dataframe

df

In [ ]:
##convert df to pickle so that it keeps all the correct dtypes

df.to_pickle('final_spotify.pkl')